# Particionamento de Dados com Apache Iceberg

O particionamento é uma técnica fundamental para otimizar consultas em grandes volumes de dados. O Apache Iceberg oferece recursos avançados como **Particionamento Oculto** e **Evolução de Particionamento**.

## Conceitos Principais:

### 1. Particionamento Oculto (Hidden Partitioning)
- O Iceberg abstrai a complexidade do particionamento
- Usuário define apenas a coluna e função de transformação (ex: `year(data_venda)`)
- Não precisa incluir colunas derivadas nas consultas
- Simplifica queries e evita erros

### 2. Evolução de Particionamento (Partition Evolution)
- Permite alterar estratégia de particionamento sem reescrever dados
- Novos dados usam novo esquema, dados antigos mantêm esquema original
- Adapta-se ao crescimento e mudanças nos padrões de consulta

## Setup do Ambiente

In [ ]:
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass

# Configuração do Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

# Sessão Spark com configuração explícita
spark = SparkSession.builder \
    .appName("IcebergPartitioning") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

## 1. Particionamento por Ano

Vamos criar uma tabela particionada por ano usando a função `year(data_venda)`. Isso permite que consultas filtradas por ano leiam apenas os arquivos relevantes.

In [ ]:
# Criar tabela particionada por ano da data de venda
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_partitioned (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (year(data_venda))
""")

### Inserindo Dados de Diferentes Anos

Vamos inserir dados de 2022, 2023 e 2024 para demonstrar como o Iceberg organiza automaticamente os dados em partições por ano.

In [ ]:
# Inserir dados de diferentes anos
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_partitioned VALUES
    (1, 'Produto A', 10, 15.5, DATE('2022-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2022-06-20')),
    (3, 'Produto C', 8, 30.0, DATE('2023-03-10')),
    (4, 'Produto D', 12, 25.0, DATE('2023-08-25')),
    (5, 'Produto E', 7, 18.5, DATE('2024-02-14'))
""")

### Visualizando os Dados

Observe que consultamos diretamente a coluna `data_venda` sem precisar especificar a partição.

In [ ]:
# Visualizar dados
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_partitioned ORDER BY data_venda").show()

### Visualizando as Partições

O Iceberg cria automaticamente partições baseadas no ano. Cada partição contém estatísticas úteis para otimização.

In [ ]:
# Ver informações sobre partições
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_partitioned.partitions").show()

### Consulta Otimizada por Partição

Esta consulta lerá apenas os arquivos da partição de 2023, demonstrando o **partition pruning**.

In [ ]:
# Consulta que aproveita particionamento (apenas dados de 2023)
spark.sql("""
    SELECT * FROM hadoop_catalog.default.vendas_partitioned 
    WHERE year(data_venda) = 2023
    ORDER BY data_venda
""").show()

### Visualizando Arquivos por Partição

Podemos ver como os arquivos estão organizados fisicamente por partição.

In [ ]:
# Ver arquivos da tabela particionada
spark.sql("SELECT file_path, partition FROM hadoop_catalog.default.vendas_partitioned.files").show(truncate=False)

## 2. Particionamento por Dias

Para dados com alta granularidade temporal, podemos usar `days(data_venda)` que particiona por dia desde a época Unix.

In [ ]:
# Criar tabela com particionamento por dias (evita conflito year/month)
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_daily (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (days(data_venda))
""")

### Inserindo Dados na Tabela Diária

Cada data única criará uma partição separada.

In [ ]:
# Inserir dados na tabela com particionamento diário
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_daily VALUES
    (1, 'Produto A', 10, 15.5, DATE('2023-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2023-01-20')),
    (3, 'Produto C', 8, 30.0, DATE('2023-02-10')),
    (4, 'Produto D', 12, 25.0, DATE('2023-02-25')),
    (5, 'Produto E', 7, 18.5, DATE('2023-03-14'))
""")

### Visualizando Partições Diárias

Observe que cada data única cria uma partição com número de dias desde 1970-01-01.

In [ ]:
# Ver partições da tabela diária
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_daily.partitions").show()

## 3. Particionamento por Bucket

O particionamento por bucket distribui dados uniformemente usando uma função hash. Útil para balanceamento de carga e joins eficientes.

In [ ]:
# Criar tabela com particionamento por bucket (alternativa para múltiplas dimensões)
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_bucket (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (bucket(4, id))
""")

### Inserindo Dados com Bucket

Os dados serão distribuídos em 4 buckets baseados no hash do campo `id`.

In [ ]:
# Inserir dados na tabela com bucket
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_bucket VALUES
    (1, 'Produto A', 10, 15.5, DATE('2023-01-15')),
    (2, 'Produto B', 5, 22.0, DATE('2023-01-20')),
    (3, 'Produto C', 8, 30.0, DATE('2023-02-10')),
    (4, 'Produto D', 12, 25.0, DATE('2023-02-25')),
    (5, 'Produto E', 7, 18.5, DATE('2023-03-14'))
""")

### Visualizando Partições por Bucket

Cada bucket (0-3) contém dados distribuídos uniformemente pelo hash do ID.

In [ ]:
# Ver partições por bucket
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_bucket.partitions").show()

## 4. Particionamento por Categoria

Particionamento por valores categóricos é útil quando temos consultas frequentes filtradas por categoria específica.

In [ ]:
# Exemplo de particionamento por categoria de produto
spark.sql("""
    CREATE TABLE IF NOT EXISTS hadoop_catalog.default.vendas_categoria (
        id INT,
        produto STRING,
        categoria STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
    PARTITIONED BY (categoria)
""")

### Inserindo Dados com Categorias

Cada categoria criará uma partição separada, otimizando consultas por categoria.

In [ ]:
# Inserir dados com categorias
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas_categoria VALUES
    (1, 'Notebook', 'Eletrônicos', 2, 2500.0, DATE('2023-01-15')),
    (2, 'Mesa', 'Móveis', 1, 800.0, DATE('2023-01-20')),
    (3, 'Smartphone', 'Eletrônicos', 3, 1200.0, DATE('2023-02-10')),
    (4, 'Cadeira', 'Móveis', 4, 300.0, DATE('2023-02-25')),
    (5, 'Tablet', 'Eletrônicos', 1, 600.0, DATE('2023-03-14'))
""")

### Consulta Otimizada por Categoria

Esta consulta lerá apenas os arquivos da partição 'Eletrônicos', ignorando dados de outras categorias.

In [ ]:
# Consultar apenas eletrônicos
spark.sql("""
    SELECT * FROM hadoop_catalog.default.vendas_categoria 
    WHERE categoria = 'Eletrônicos'
    ORDER BY data_venda
""").show()

### Visualizando Partições por Categoria

Podemos ver as partições criadas para cada categoria única.

In [ ]:
# Ver partições por categoria
spark.sql("SELECT * FROM hadoop_catalog.default.vendas_categoria.partitions").show()